In [106]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from joblib import dump
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [107]:
df1 = pd.read_csv("undark.csv")
df2 = pd.read_csv("dark_patterns.csv")
df1

,Link,Pattern String,Deceptive?
0,https://www.fashionnova.com/,Fashion Nova | Fashion Online For Women | Affo...,0
1,https://www.gymshark.com/,Gymshark Official Store - Shop Gym Clothes & W...,0
2,https://zanerobe.com/,ZANEROBE - Considered and contemporary men’s a...,0
3,https://colourpop.com/,ColourPop Cosmetics Skip to content ✈ FREE WOR...,0
4,https://www.aloyoga.com/,"Alo Yoga | Yoga leggings, clothes, and accesso...",0
...,...,...,...
715,https://www.plumbworld.co.uk/ceramica-curved-b...,Offer ends in 03 days 19 hours : 54 mins : 22 ...,1
716,https://www.prettylittlething.us/freyana-rose-...,35% OFF EVERYTHING* EXCL SALE & BEAUTY ENTER: ...,1
717,https://www.prettylittlething.us/freyana-rose-...,"HURRY, ORDER WITHIN 16h 59m 06s TO GET 35% OFF...",1
718,https://www.proflowers.com/product/premium-all...,LAST CHANCE TO SAVE... enter your email to Sav...,1


In [108]:
#Remove null strings in Pattern String Column
df1 = df1[pd.notnull(df1["Pattern String"])]

df1 = df1[df1["Deceptive?"] == 0]
df1["Deceptive?"] = "Not Dark"
col = ["Pattern String", "Deceptive?"]
df1= df1[col]
df1

,Pattern String,Deceptive?
0,Fashion Nova | Fashion Online For Women | Affo...,Not Dark
1,Gymshark Official Store - Shop Gym Clothes & W...,Not Dark
2,ZANEROBE - Considered and contemporary men’s a...,Not Dark
3,ColourPop Cosmetics Skip to content ✈ FREE WOR...,Not Dark
4,"Alo Yoga | Yoga leggings, clothes, and accesso...",Not Dark
...,...,...
650,"Herrschners, INC. — The Ultimate Source for Cr...",Not Dark
651,Body Wash & Deodorants | Personal Care | Ivory...,Not Dark
652,Body Wash & Deodorants | Personal Care | Ivory...,Not Dark
653,"Women's Designer Shoes - Golden Goose, Veja, J...",Not Dark


In [109]:
df2

,Pattern String,Comment,Pattern Category,Pattern Type,Where in website?,Deceptive?,Website Page
0,Collin P. from Grandview Missouri just bought ...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://alaindupetit.com/collections/all-suits...
1,"Faith in Glendale, United States purchased a C...",Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bonescoffee.com/products/strawberry-ch...
2,Sharmeen Atif From Karachi just bought Stylish...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://brandsego.com/collections/under-rs-99/...
3,9 people are viewing this.,Product detail,Social Proof,Activity Notification,Product Page,No,https://brightechshop.com/products/ambience-so...
4,5338 people viewed this in the last hour,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bumpboxes.com/
...,...,...,...,...,...,...,...
1813,$132.90 $99.00,Website adds free items to show discount,Misdirection,Visual Interference,Cart Page,No,https://www.planetofthevapes.com/products/plan...
1814,This offer is only VALID if you add to cart now!,Popup asking you to buy more,Misdirection,Visual Interference,Product Page,No,https://www.rockymountainoils.com/single-essen...
1815,NaN,Deterministic draw. Always give you the prize ...,Misdirection,Visual Interference,Product Page,Yes,https://www.sammydress.com/
1816,NaN,Shows you prices in the popup based on your cu...,Misdirection,Visual Interference,Product Page,No,https://www.shoedazzle.com/products/FEELIN-A-L...


In [110]:
df2 = df2[pd.notnull(df2["Pattern String"])]
df2["Deceptive?"] = "Dark"
col = ["Pattern String", "Deceptive?"]
df2 = df2[col]
df2

C:\Users\death\AppData\Local\Temp\ipykernel_24616\1550006063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Deceptive?"] = "Dark"


,Pattern String,Deceptive?
0,Collin P. from Grandview Missouri just bought ...,Dark
1,"Faith in Glendale, United States purchased a C...",Dark
2,Sharmeen Atif From Karachi just bought Stylish...,Dark
3,9 people are viewing this.,Dark
4,5338 people viewed this in the last hour,Dark
...,...,...
1810,TWO FREE PILLOWS AND 30% OFF WITH PROMO CODE,Dark
1812,Compare at,Dark
1813,$132.90 $99.00,Dark
1814,This offer is only VALID if you add to cart now!,Dark


In [113]:
final_df = pd.concat([df1, df2])
final_df

,Pattern String,Deceptive?
0,Fashion Nova | Fashion Online For Women | Affo...,Not Dark
1,Gymshark Official Store - Shop Gym Clothes & W...,Not Dark
2,ZANEROBE - Considered and contemporary men’s a...,Not Dark
3,ColourPop Cosmetics Skip to content ✈ FREE WOR...,Not Dark
4,"Alo Yoga | Yoga leggings, clothes, and accesso...",Not Dark
...,...,...
1810,TWO FREE PILLOWS AND 30% OFF WITH PROMO CODE,Dark
1812,Compare at,Dark
1813,$132.90 $99.00,Dark
1814,This offer is only VALID if you add to cart now!,Dark


In [114]:
X_train, X_test, y_train, y_test = train_test_split(
    final_df['Pattern String'], final_df["Deceptive?"], train_size = .7)
#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [115]:
# Create TfidfVectorizer
vectorizer = TfidfVectorizer()

# Transform training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform test data
X_test_tfidf = vectorizer.transform(X_test)

# Create a dictionary to hold the classifiers
classifiers = {
    'Bernoulli Naive Bayes': BernoulliNB(),
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}



# Create a dictionary to hold the classifiers
classifiers = {
    'Bernoulli Naive Bayes': BernoulliNB(),
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

# Create an array to store accuracies
accuracies = []

# Iterate through classifiers
for name, clf in classifiers.items():
    # Create a pipeline with vectorizer and classifier
    text_clf = make_pipeline(TfidfVectorizer(), clf)
    
    # Define the parameter grid for grid search
    parameters = {
        'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
        'tfidfvectorizer__use_idf': (True, False),
    }

    # Adjust parameters for Logistic Regression
    if name == 'Logistic Regression':
        parameters['logisticregression__C'] = [0.001, 0.01, 0.1, 1, 10, 100]

    # Perform grid search with cross-validation
    grid_search = GridSearchCV(text_clf, parameters, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Print the best parameters found by grid search
    print(f"Best parameters for {name}: ", grid_search.best_params_)

    # Fit the model with the best parameters
    best_clf = grid_search.best_estimator_
    
    # Make predictions on the test set
    y_pred = best_clf.predict(X_test)

    # Calculate accuracy
    acc = metrics.accuracy_score(y_test, y_pred)
    
    # Save the result
    accuracies.append((name, acc))

# Print the results
for name, acc in accuracies:
    print(f'{name} Accuracy: {acc:.4f}')

Best parameters for Bernoulli Naive Bayes:  {'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__use_idf': True}
Best parameters for Logistic Regression:  {'logisticregression__C': 100, 'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__use_idf': True}
Best parameters for Support Vector Machine:  {'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__use_idf': True}
Best parameters for Random Forest:  {'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__use_idf': False}
Best parameters for Gradient Boosting:  {'tfidfvectorizer__ngram_range': (1, 1), 'tfidfvectorizer__use_idf': True}
Bernoulli Naive Bayes Accuracy: 0.9580
Logistic Regression Accuracy: 0.9895
Support Vector Machine Accuracy: 0.9860
Random Forest Accuracy: 0.9790
Gradient Boosting Accuracy: 0.9720


In [116]:
# Evaluate the best performing classifier on the test set
best_classifier_name, best_classifier_acc = max(accuracies, key=lambda x: x[1])
print(f"\nBest Performing Classifier: {best_classifier_name}, Accuracy: {best_classifier_acc:.4f}")


Best Performing Classifier: Logistic Regression, Accuracy: 0.9895


In [105]:
# Save the best model to disk
best_classifier_name, best_classifier_acc = max(accuracies, key=lambda x: x[1])
best_model = make_pipeline(vectorizer, classifiers[best_classifier_name])
dump(best_model, 'best_model_presenceClassifier.joblib')

['best_model.joblib']